In [1]:
import os
from google.cloud import storage
from google.oauth2 import service_account
from ast import literal_eval
import pandas as pd
from io import BytesIO, StringIO
import pickle
import pandas as pd

# Environment

In [2]:
# settings.py
import os
from dotenv import load_dotenv
load_dotenv()

PROJECT_ID = os.environ.get("PROJECT_ID")
PROJECT_NAME = os.environ.get("PROJECT_NAME")
BUCKET_NAME = os.environ.get("BUCKET_NAME")
PROJECT_ID, BUCKET_NAME, PROJECT_NAME
credential = service_account.Credentials.from_service_account_info(literal_eval(os.environ.get("CREDENTIAL")))
client = storage.Client(project=PROJECT_NAME, credentials=credential)

# Load Model

In [ ]:
def load_model_from_storage(model_path):
    bucket = client.get_bucket(BUCKET_NAME)
    blob = bucket.get_blob(model_path) # File path @CloudStorage
    # blob.download_to_filename("data/diabetes_gcs.csv") # Download to local
    gcs_model = pickle.load(BytesIO(blob.download_as_string()))
    return gcs_model

model = load_model_from_storage("model/logreg.pkl")

# Read Dataset

In [ ]:
def read_dataset(input_file):
    bucket = client.get_bucket(BUCKET_NAME)
    blob = bucket.get_blob(input_file) # File path @CloudStorage
    # blob.download_to_filename("data/diabetes_gcs.csv") # Download to local
    df = pd.read_csv(BytesIO(blob.download_as_string()))
    return df
df = read_dataset("input/diabetes.csv")
df.head()

# Save to Storage

In [ ]:
df["prediction"] = model.predict(df)

In [ ]:
def save_result_to_storage(df, file_name):
    df_new = df.to_csv(index=False)
    content_type = "text/csv" if ".csv" in file_name else "text/plain"
    bucket = client.get_bucket(BUCKET_NAME)
    blob = bucket.blob(file_name) # File path @CloudStorage
    blob.upload_from_string(df_new, content_type=content_type) # File path @Local

save_result_to_storage(df, "output/result.csv")

# Calling Cloud Function

In [3]:
from google.auth.transport.requests import AuthorizedSession

URL = os.environ.get("GCF_URL")
CREDENTIAL_PATH = os.environ.get("CREDENTIAL_PATH")
creds = service_account.IDTokenCredentials.from_service_account_file(CREDENTIAL_PATH, target_audience=URL)
authed_session = AuthorizedSession(creds)

# make authenticated request and print the response, status_code
resp = authed_session.get(URL)
print(resp.status_code)

200
